In [1]:
print("""
@File         : ch02_finding_and_wrangling_time_series_data.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-03-08 23:25:05
@Email        : cuixuanstephen@gmail.com
@Description  : 时间序列的发现与整理
""")


@File         : ch02_finding_and_wrangling_time_series_data.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-03-08 23:25:05
@Email        : cuixuanstephen@gmail.com
@Description  : 时间序列的发现与整理

